# Data

In [1]:
from bpe import Encoder
import pickle as pkl

with open('data/vocab/de.pkl', 'rb') as f:
    de_vocab = pkl.load(f)

with open('data/vocab/en.pkl', 'rb') as f:
    en_vocab = pkl.load(f)

In [2]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

BATCH_SIZE = 32
PAD_DE = de_vocab.word_vocab['__pad']
BOS_DE = 16384
EOS_DE = 16385

PAD_EN = en_vocab.word_vocab['__pad']
BOS_EN = 16384
EOS_EN = 16385

def generate_batch(data_batch):
    de_batch, en_batch = [], []
    for (de_item, en_item) in data_batch:
        de_batch.append(torch.cat([torch.tensor([BOS_DE]), de_item, torch.tensor([EOS_DE])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_EN]), en_item, torch.tensor([EOS_EN])], dim=0))
    de_batch = pad_sequence(de_batch, padding_value=PAD_DE)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN)
    return de_batch.to('cuda'), en_batch.to('cuda')

# Train

In [3]:
import torch
import torch.nn as nn
import math
from model import Transformer

vsize1 = 16386
vsize2 = 16386
model = Transformer(d_model=512, nhead=8, 
                    nlayers_enc=3, nlayers_dec=3, d_fforward=1024, # !
                    vsize1=vsize1, vsize2=vsize2, dropout=0.1).to('cuda')

In [4]:
optimizer = torch.optim.Adam(model.parameters(),  betas=(0.9, 0.98), eps=1e-9, lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_EN)

for epoch in range(3):
    for pack in range(45):
        with open('data/train/' + str(pack) + '.pkl', 'rb') as f: 
            X = pkl.load(f)
        dloader = DataLoader(X, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=generate_batch)
        for i, (seq1, seq2) in enumerate(dloader):
            tgt_in = seq2[:-1]
            pred = model(seq1, tgt_in)

            tgt_out = seq2[1:]
            loss = criterion(pred.reshape(-1, 16386), tgt_out.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 128 == 0:
                print('Epoch {} Pack {} Step {} Loss {}'.format(epoch, pack, i, loss.detach()))

Epoch 0 Pack 0 Step 0 Loss 9.873409271240234
Epoch 0 Pack 0 Step 128 Loss 5.703582763671875
Epoch 0 Pack 0 Step 256 Loss 5.347036361694336
Epoch 0 Pack 0 Step 384 Loss 4.948539733886719
Epoch 0 Pack 0 Step 512 Loss 5.0226945877075195
Epoch 0 Pack 0 Step 640 Loss 4.837382793426514
Epoch 0 Pack 0 Step 768 Loss 4.536397933959961
Epoch 0 Pack 0 Step 896 Loss 4.833409309387207
Epoch 0 Pack 0 Step 1024 Loss 4.883787631988525
Epoch 0 Pack 0 Step 1152 Loss 4.7588043212890625
Epoch 0 Pack 0 Step 1280 Loss 4.460808753967285
Epoch 0 Pack 0 Step 1408 Loss 4.615694522857666
Epoch 0 Pack 0 Step 1536 Loss 4.421431541442871
Epoch 0 Pack 0 Step 1664 Loss 4.83182954788208
Epoch 0 Pack 0 Step 1792 Loss 4.452064037322998
Epoch 0 Pack 0 Step 1920 Loss 4.344852924346924
Epoch 0 Pack 0 Step 2048 Loss 4.481421947479248
Epoch 0 Pack 0 Step 2176 Loss 4.410529136657715
Epoch 0 Pack 0 Step 2304 Loss 4.439087867736816
Epoch 0 Pack 0 Step 2432 Loss 4.146240234375
Epoch 0 Pack 0 Step 2560 Loss 4.263702392578125
Epoc

Epoch 0 Pack 6 Step 2816 Loss 3.1581270694732666
Epoch 0 Pack 6 Step 2944 Loss 2.9759433269500732
Epoch 0 Pack 6 Step 3072 Loss 3.6525561809539795
Epoch 0 Pack 7 Step 0 Loss 4.021973133087158
Epoch 0 Pack 7 Step 128 Loss 3.5031416416168213
Epoch 0 Pack 7 Step 256 Loss 3.7514448165893555
Epoch 0 Pack 7 Step 384 Loss 3.4230360984802246
Epoch 0 Pack 7 Step 512 Loss 3.541544198989868
Epoch 0 Pack 7 Step 640 Loss 3.3801026344299316
Epoch 0 Pack 7 Step 768 Loss 3.2306618690490723
Epoch 0 Pack 7 Step 896 Loss 3.312650203704834
Epoch 0 Pack 7 Step 1024 Loss 3.1554369926452637
Epoch 0 Pack 7 Step 1152 Loss 3.443466901779175
Epoch 0 Pack 7 Step 1280 Loss 3.505019426345825
Epoch 0 Pack 7 Step 1408 Loss 3.6333301067352295
Epoch 0 Pack 7 Step 1536 Loss 3.954744338989258
Epoch 0 Pack 7 Step 1664 Loss 3.274282693862915
Epoch 0 Pack 7 Step 1792 Loss 3.372851848602295
Epoch 0 Pack 7 Step 1920 Loss 3.50038743019104
Epoch 0 Pack 7 Step 2048 Loss 3.4378020763397217
Epoch 0 Pack 7 Step 2176 Loss 3.51464915

Epoch 0 Pack 13 Step 2048 Loss 3.4542295932769775
Epoch 0 Pack 13 Step 2176 Loss 3.588744878768921
Epoch 0 Pack 13 Step 2304 Loss 3.5646538734436035
Epoch 0 Pack 13 Step 2432 Loss 3.4449422359466553
Epoch 0 Pack 13 Step 2560 Loss 3.689298629760742
Epoch 0 Pack 13 Step 2688 Loss 3.495553493499756
Epoch 0 Pack 13 Step 2816 Loss 3.5120632648468018
Epoch 0 Pack 13 Step 2944 Loss 3.3467860221862793
Epoch 0 Pack 13 Step 3072 Loss 3.3786513805389404
Epoch 0 Pack 14 Step 0 Loss 3.519282579421997
Epoch 0 Pack 14 Step 128 Loss 3.7042438983917236
Epoch 0 Pack 14 Step 256 Loss 3.47883939743042
Epoch 0 Pack 14 Step 384 Loss 3.630617141723633
Epoch 0 Pack 14 Step 512 Loss 3.6518163681030273
Epoch 0 Pack 14 Step 640 Loss 3.5135929584503174
Epoch 0 Pack 14 Step 768 Loss 3.5150177478790283
Epoch 0 Pack 14 Step 896 Loss 3.636547565460205
Epoch 0 Pack 14 Step 1024 Loss 3.3938870429992676
Epoch 0 Pack 14 Step 1152 Loss 3.388380527496338
Epoch 0 Pack 14 Step 1280 Loss 3.5567574501037598
Epoch 0 Pack 14 Ste

Epoch 0 Pack 20 Step 1152 Loss 3.318918228149414
Epoch 0 Pack 20 Step 1280 Loss 3.2824344635009766
Epoch 0 Pack 20 Step 1408 Loss 3.1669516563415527
Epoch 0 Pack 20 Step 1536 Loss 3.5474846363067627
Epoch 0 Pack 20 Step 1664 Loss 3.4505615234375
Epoch 0 Pack 20 Step 1792 Loss 3.4203922748565674
Epoch 0 Pack 20 Step 1920 Loss 3.478520393371582
Epoch 0 Pack 20 Step 2048 Loss 3.44301176071167
Epoch 0 Pack 20 Step 2176 Loss 3.1585326194763184
Epoch 0 Pack 20 Step 2304 Loss 3.1089985370635986
Epoch 0 Pack 20 Step 2432 Loss 3.394350051879883
Epoch 0 Pack 20 Step 2560 Loss 3.432656764984131
Epoch 0 Pack 20 Step 2688 Loss 3.3194992542266846
Epoch 0 Pack 20 Step 2816 Loss 3.051650047302246
Epoch 0 Pack 20 Step 2944 Loss 3.5699303150177
Epoch 0 Pack 20 Step 3072 Loss 3.4044671058654785
Epoch 0 Pack 21 Step 0 Loss 3.3037047386169434
Epoch 0 Pack 21 Step 128 Loss 3.4106662273406982
Epoch 0 Pack 21 Step 256 Loss 3.624248743057251
Epoch 0 Pack 21 Step 384 Loss 3.5174429416656494
Epoch 0 Pack 21 Step

Epoch 0 Pack 27 Step 256 Loss 2.7176923751831055
Epoch 0 Pack 27 Step 384 Loss 2.8408989906311035
Epoch 0 Pack 27 Step 512 Loss 2.7578399181365967
Epoch 0 Pack 27 Step 640 Loss 2.8743414878845215
Epoch 0 Pack 27 Step 768 Loss 2.820342779159546
Epoch 0 Pack 27 Step 896 Loss 2.9693353176116943
Epoch 0 Pack 27 Step 1024 Loss 2.979782819747925
Epoch 0 Pack 27 Step 1152 Loss 2.910003423690796
Epoch 0 Pack 27 Step 1280 Loss 2.775486946105957
Epoch 0 Pack 27 Step 1408 Loss 2.857760190963745
Epoch 0 Pack 27 Step 1536 Loss 2.8561079502105713
Epoch 0 Pack 27 Step 1664 Loss 2.6900367736816406
Epoch 0 Pack 27 Step 1792 Loss 2.937239408493042
Epoch 0 Pack 27 Step 1920 Loss 2.99137020111084
Epoch 0 Pack 27 Step 2048 Loss 2.8039562702178955
Epoch 0 Pack 27 Step 2176 Loss 2.9405486583709717
Epoch 0 Pack 27 Step 2304 Loss 2.841306447982788
Epoch 0 Pack 27 Step 2432 Loss 2.762047290802002
Epoch 0 Pack 27 Step 2560 Loss 2.6758782863616943
Epoch 0 Pack 27 Step 2688 Loss 2.665541648864746
Epoch 0 Pack 27 S

...

Epoch 2 Pack 11 Step 2304 Loss 2.8689732551574707
Epoch 2 Pack 11 Step 2432 Loss 2.9566264152526855
Epoch 2 Pack 11 Step 2560 Loss 2.9964637756347656
Epoch 2 Pack 11 Step 2688 Loss 3.178823232650757
Epoch 2 Pack 11 Step 2816 Loss 2.7540481090545654
Epoch 2 Pack 11 Step 2944 Loss 2.8606419563293457
Epoch 2 Pack 11 Step 3072 Loss 3.034844160079956
Epoch 2 Pack 12 Step 0 Loss 3.004805326461792
Epoch 2 Pack 12 Step 128 Loss 2.980705499649048
Epoch 2 Pack 12 Step 256 Loss 3.012934684753418
Epoch 2 Pack 12 Step 384 Loss 3.0729870796203613
Epoch 2 Pack 12 Step 512 Loss 2.9209625720977783
Epoch 2 Pack 12 Step 640 Loss 2.979959011077881
Epoch 2 Pack 12 Step 768 Loss 3.4900646209716797
Epoch 2 Pack 12 Step 896 Loss 3.171614170074463
Epoch 2 Pack 12 Step 1024 Loss 3.0066192150115967
Epoch 2 Pack 12 Step 1152 Loss 2.657050609588623
Epoch 2 Pack 12 Step 1280 Loss 3.1175642013549805
Epoch 2 Pack 12 Step 1408 Loss 3.1069982051849365
Epoch 2 Pack 12 Step 1536 Loss 2.793184280395508
Epoch 2 Pack 12 Ste

Epoch 2 Pack 18 Step 1280 Loss 3.0374624729156494
Epoch 2 Pack 18 Step 1408 Loss 3.0954651832580566
Epoch 2 Pack 18 Step 1536 Loss 3.0448601245880127
Epoch 2 Pack 18 Step 1664 Loss 3.098564863204956
Epoch 2 Pack 18 Step 1792 Loss 3.0365631580352783
Epoch 2 Pack 18 Step 1920 Loss 3.1714494228363037
Epoch 2 Pack 18 Step 2048 Loss 2.9213483333587646
Epoch 2 Pack 18 Step 2176 Loss 2.9647021293640137
Epoch 2 Pack 18 Step 2304 Loss 3.0062897205352783
Epoch 2 Pack 18 Step 2432 Loss 2.9816877841949463
Epoch 2 Pack 18 Step 2560 Loss 3.0201587677001953
Epoch 2 Pack 18 Step 2688 Loss 3.076098918914795
Epoch 2 Pack 18 Step 2816 Loss 2.927645444869995
Epoch 2 Pack 18 Step 2944 Loss 3.205322742462158
Epoch 2 Pack 18 Step 3072 Loss 3.245198965072632
Epoch 2 Pack 19 Step 0 Loss 3.0680530071258545
Epoch 2 Pack 19 Step 128 Loss 2.900499105453491
Epoch 2 Pack 19 Step 256 Loss 3.065136194229126
Epoch 2 Pack 19 Step 384 Loss 3.0670762062072754
Epoch 2 Pack 19 Step 512 Loss 3.0261130332946777
Epoch 2 Pack 1

Epoch 2 Pack 25 Step 256 Loss 2.3389530181884766
Epoch 2 Pack 25 Step 384 Loss 2.4801254272460938
Epoch 2 Pack 25 Step 512 Loss 2.6606829166412354
Epoch 2 Pack 25 Step 640 Loss 2.6703195571899414
Epoch 2 Pack 25 Step 768 Loss 2.3901097774505615
Epoch 2 Pack 25 Step 896 Loss 2.5826659202575684
Epoch 2 Pack 25 Step 1024 Loss 2.4410645961761475
Epoch 2 Pack 25 Step 1152 Loss 2.753307342529297
Epoch 2 Pack 25 Step 1280 Loss 2.5121381282806396
Epoch 2 Pack 25 Step 1408 Loss 2.459074020385742
Epoch 2 Pack 25 Step 1536 Loss 2.2139453887939453
Epoch 2 Pack 25 Step 1664 Loss 2.4850804805755615
Epoch 2 Pack 25 Step 1792 Loss 2.2299578189849854
Epoch 2 Pack 25 Step 1920 Loss 2.3428075313568115
Epoch 2 Pack 25 Step 2048 Loss 2.615093946456909
Epoch 2 Pack 25 Step 2176 Loss 2.297024726867676
Epoch 2 Pack 25 Step 2304 Loss 2.583113670349121
Epoch 2 Pack 25 Step 2432 Loss 2.3296308517456055
Epoch 2 Pack 25 Step 2560 Loss 2.549381732940674
Epoch 2 Pack 25 Step 2688 Loss 2.5063090324401855
Epoch 2 Pack

Epoch 2 Pack 31 Step 2432 Loss 1.9977571964263916
Epoch 2 Pack 31 Step 2560 Loss 2.1221868991851807
Epoch 2 Pack 31 Step 2688 Loss 2.062764883041382
Epoch 2 Pack 31 Step 2816 Loss 1.975082516670227
Epoch 2 Pack 31 Step 2944 Loss 2.3953046798706055
Epoch 2 Pack 31 Step 3072 Loss 2.1393966674804688
Epoch 2 Pack 32 Step 0 Loss 2.2951884269714355
Epoch 2 Pack 32 Step 128 Loss 2.0537760257720947
Epoch 2 Pack 32 Step 256 Loss 2.0810904502868652
Epoch 2 Pack 32 Step 384 Loss 2.4164352416992188
Epoch 2 Pack 32 Step 512 Loss 2.176671028137207
Epoch 2 Pack 32 Step 640 Loss 2.141800880432129
Epoch 2 Pack 32 Step 768 Loss 2.270402193069458
Epoch 2 Pack 32 Step 896 Loss 2.2637147903442383
Epoch 2 Pack 32 Step 1024 Loss 2.1298000812530518
Epoch 2 Pack 32 Step 1152 Loss 2.2792301177978516
Epoch 2 Pack 32 Step 1280 Loss 2.119093656539917
Epoch 2 Pack 32 Step 1408 Loss 2.1892940998077393
Epoch 2 Pack 32 Step 1536 Loss 2.437387704849243
Epoch 2 Pack 32 Step 1664 Loss 2.2909748554229736
Epoch 2 Pack 32 S

Epoch 2 Pack 38 Step 1408 Loss 2.1686851978302
Epoch 2 Pack 38 Step 1536 Loss 2.2588796615600586
Epoch 2 Pack 38 Step 1664 Loss 2.33303165435791
Epoch 2 Pack 38 Step 1792 Loss 2.228813648223877
Epoch 2 Pack 38 Step 1920 Loss 2.2827517986297607
Epoch 2 Pack 38 Step 2048 Loss 2.152411699295044
Epoch 2 Pack 38 Step 2176 Loss 2.198380947113037
Epoch 2 Pack 38 Step 2304 Loss 2.075361967086792
Epoch 2 Pack 38 Step 2432 Loss 2.1700315475463867
Epoch 2 Pack 38 Step 2560 Loss 1.938154935836792
Epoch 2 Pack 38 Step 2688 Loss 2.5813534259796143
Epoch 2 Pack 38 Step 2816 Loss 2.3069119453430176
Epoch 2 Pack 38 Step 2944 Loss 2.108241081237793
Epoch 2 Pack 38 Step 3072 Loss 2.0145041942596436
Epoch 2 Pack 39 Step 0 Loss 2.513838291168213
Epoch 2 Pack 39 Step 128 Loss 2.382711410522461
Epoch 2 Pack 39 Step 256 Loss 2.5366828441619873
Epoch 2 Pack 39 Step 384 Loss 2.32966685295105
Epoch 2 Pack 39 Step 512 Loss 2.084190845489502
Epoch 2 Pack 39 Step 640 Loss 2.1999850273132324
Epoch 2 Pack 39 Step 768

In [5]:
torch.save(model.state_dict(), 'graph/2/model1')

In [6]:
optimizer = torch.optim.Adam(model.parameters(),  betas=(0.9, 0.98), eps=1e-9, lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_EN)

for epoch in range(3):
    for pack in range(45):
        with open('data/train/' + str(pack) + '.pkl', 'rb') as f: 
            X = pkl.load(f)
        dloader = DataLoader(X, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=generate_batch)
        for i, (seq1, seq2) in enumerate(dloader):
            tgt_in = seq2[:-1]
            pred = model(seq1, tgt_in)

            tgt_out = seq2[1:]
            loss = criterion(pred.reshape(-1, 16386), tgt_out.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 128 == 0:
                print('Epoch {} Pack {} Step {} Loss {}'.format(epoch, pack, i, loss.detach()))

Epoch 0 Pack 0 Step 0 Loss 4.417440891265869
Epoch 0 Pack 0 Step 128 Loss 3.8365695476531982
Epoch 0 Pack 0 Step 256 Loss 3.492206573486328
Epoch 0 Pack 0 Step 384 Loss 3.3157830238342285
Epoch 0 Pack 0 Step 512 Loss 3.1608784198760986
Epoch 0 Pack 0 Step 640 Loss 3.328845739364624
Epoch 0 Pack 0 Step 768 Loss 3.2560198307037354
Epoch 0 Pack 0 Step 896 Loss 3.3472869396209717
Epoch 0 Pack 0 Step 1024 Loss 3.067838668823242
Epoch 0 Pack 0 Step 1152 Loss 2.948084831237793
Epoch 0 Pack 0 Step 1280 Loss 3.1551241874694824
Epoch 0 Pack 0 Step 1408 Loss 3.042832374572754
Epoch 0 Pack 0 Step 1536 Loss 3.1378180980682373
Epoch 0 Pack 0 Step 1664 Loss 3.083085298538208
Epoch 0 Pack 0 Step 1792 Loss 3.0768184661865234
Epoch 0 Pack 0 Step 1920 Loss 3.239753007888794
Epoch 0 Pack 0 Step 2048 Loss 2.9346628189086914
Epoch 0 Pack 0 Step 2176 Loss 2.9985694885253906
Epoch 0 Pack 0 Step 2304 Loss 3.18292498588562
Epoch 0 Pack 0 Step 2432 Loss 3.172661304473877
Epoch 0 Pack 0 Step 2560 Loss 2.761013269

Epoch 0 Pack 6 Step 2688 Loss 3.0105860233306885
Epoch 0 Pack 6 Step 2816 Loss 2.7973577976226807
Epoch 0 Pack 6 Step 2944 Loss 3.0582940578460693
Epoch 0 Pack 6 Step 3072 Loss 2.821915864944458
Epoch 0 Pack 7 Step 0 Loss 2.947075605392456
Epoch 0 Pack 7 Step 128 Loss 2.9673712253570557
Epoch 0 Pack 7 Step 256 Loss 3.0373096466064453
Epoch 0 Pack 7 Step 384 Loss 3.0272300243377686
Epoch 0 Pack 7 Step 512 Loss 2.843409299850464
Epoch 0 Pack 7 Step 640 Loss 3.1881279945373535
Epoch 0 Pack 7 Step 768 Loss 3.381007194519043
Epoch 0 Pack 7 Step 896 Loss 3.3163952827453613
Epoch 0 Pack 7 Step 1024 Loss 3.147826671600342
Epoch 0 Pack 7 Step 1152 Loss 3.1495938301086426
Epoch 0 Pack 7 Step 1280 Loss 2.9309985637664795
Epoch 0 Pack 7 Step 1408 Loss 2.8709654808044434
Epoch 0 Pack 7 Step 1536 Loss 3.2405354976654053
Epoch 0 Pack 7 Step 1664 Loss 2.995363473892212
Epoch 0 Pack 7 Step 1792 Loss 3.187443971633911
Epoch 0 Pack 7 Step 1920 Loss 2.9487762451171875
Epoch 0 Pack 7 Step 2048 Loss 3.41425

Epoch 0 Pack 13 Step 3072 Loss 2.6557631492614746
Epoch 0 Pack 14 Step 0 Loss 2.946289539337158
Epoch 0 Pack 14 Step 128 Loss 3.400587797164917
Epoch 0 Pack 14 Step 256 Loss 2.8975038528442383
Epoch 0 Pack 14 Step 384 Loss 3.102246046066284
Epoch 0 Pack 14 Step 512 Loss 3.169729471206665
Epoch 0 Pack 14 Step 640 Loss 3.1077332496643066
Epoch 0 Pack 14 Step 768 Loss 2.803215503692627
Epoch 0 Pack 14 Step 896 Loss 3.121817111968994
Epoch 0 Pack 14 Step 1024 Loss 3.120652914047241
Epoch 0 Pack 14 Step 1152 Loss 3.1203525066375732
Epoch 0 Pack 14 Step 1280 Loss 2.9488391876220703
Epoch 0 Pack 14 Step 1408 Loss 2.8709304332733154
Epoch 0 Pack 14 Step 1536 Loss 2.5551934242248535
Epoch 0 Pack 14 Step 1664 Loss 2.735529661178589
Epoch 0 Pack 14 Step 1792 Loss 3.251875638961792
Epoch 0 Pack 14 Step 1920 Loss 2.9370627403259277
Epoch 0 Pack 14 Step 2048 Loss 3.334012031555176
Epoch 0 Pack 14 Step 2176 Loss 2.980745792388916
Epoch 0 Pack 14 Step 2304 Loss 2.8537514209747314
Epoch 0 Pack 14 Step 

Epoch 0 Pack 20 Step 2048 Loss 3.3653390407562256
Epoch 0 Pack 20 Step 2176 Loss 3.4302730560302734
Epoch 0 Pack 20 Step 2304 Loss 2.6888649463653564
Epoch 0 Pack 20 Step 2432 Loss 2.8369035720825195
Epoch 0 Pack 20 Step 2560 Loss 3.1499922275543213
Epoch 0 Pack 20 Step 2688 Loss 3.0454530715942383
Epoch 0 Pack 20 Step 2816 Loss 2.949549674987793
Epoch 0 Pack 20 Step 2944 Loss 3.2603261470794678
Epoch 0 Pack 20 Step 3072 Loss 2.9574296474456787
Epoch 0 Pack 21 Step 0 Loss 2.9594168663024902
Epoch 0 Pack 21 Step 128 Loss 3.0333852767944336
Epoch 0 Pack 21 Step 256 Loss 3.024507999420166
Epoch 0 Pack 21 Step 384 Loss 3.0007615089416504
Epoch 0 Pack 21 Step 512 Loss 3.1700727939605713
Epoch 0 Pack 21 Step 640 Loss 2.858325242996216
Epoch 0 Pack 21 Step 768 Loss 3.060889720916748
Epoch 0 Pack 21 Step 896 Loss 3.114935874938965
Epoch 0 Pack 21 Step 1024 Loss 3.0277905464172363
Epoch 0 Pack 21 Step 1152 Loss 3.0945394039154053
Epoch 0 Pack 21 Step 1280 Loss 3.1784932613372803
Epoch 0 Pack 21

Epoch 0 Pack 27 Step 1024 Loss 2.585777759552002
Epoch 0 Pack 27 Step 1152 Loss 2.201251268386841
Epoch 0 Pack 27 Step 1280 Loss 2.406629800796509
Epoch 0 Pack 27 Step 1408 Loss 2.2084975242614746
Epoch 0 Pack 27 Step 1536 Loss 2.0617291927337646
Epoch 0 Pack 27 Step 1664 Loss 2.4751152992248535
Epoch 0 Pack 27 Step 1792 Loss 2.330845594406128
Epoch 0 Pack 27 Step 1920 Loss 2.0739893913269043
Epoch 0 Pack 27 Step 2048 Loss 2.2825255393981934
Epoch 0 Pack 27 Step 2176 Loss 2.458808660507202
Epoch 0 Pack 27 Step 2304 Loss 2.350100517272949
Epoch 0 Pack 27 Step 2432 Loss 2.464129686355591
Epoch 0 Pack 27 Step 2560 Loss 2.3051552772521973
Epoch 0 Pack 27 Step 2688 Loss 2.309837818145752
Epoch 0 Pack 27 Step 2816 Loss 2.262993097305298
Epoch 0 Pack 27 Step 2944 Loss 2.405740976333618
Epoch 0 Pack 27 Step 3072 Loss 2.220463275909424
Epoch 0 Pack 28 Step 0 Loss 2.448219060897827
Epoch 0 Pack 28 Step 128 Loss 2.4204328060150146
Epoch 0 Pack 28 Step 256 Loss 2.5030481815338135
Epoch 0 Pack 28 S

...

Epoch 2 Pack 11 Step 3072 Loss 2.968642234802246
Epoch 2 Pack 12 Step 0 Loss 3.0816166400909424
Epoch 2 Pack 12 Step 128 Loss 2.7241222858428955
Epoch 2 Pack 12 Step 256 Loss 2.7684528827667236
Epoch 2 Pack 12 Step 384 Loss 2.813037872314453
Epoch 2 Pack 12 Step 512 Loss 2.987795352935791
Epoch 2 Pack 12 Step 640 Loss 2.9332170486450195
Epoch 2 Pack 12 Step 768 Loss 3.1678335666656494
Epoch 2 Pack 12 Step 896 Loss 2.805642604827881
Epoch 2 Pack 12 Step 1024 Loss 2.750128984451294
Epoch 2 Pack 12 Step 1152 Loss 3.1936800479888916
Epoch 2 Pack 12 Step 1280 Loss 2.7055470943450928
Epoch 2 Pack 12 Step 1408 Loss 2.9310028553009033
Epoch 2 Pack 12 Step 1536 Loss 3.2157936096191406
Epoch 2 Pack 12 Step 1664 Loss 2.8975934982299805
Epoch 2 Pack 12 Step 1792 Loss 3.2463114261627197
Epoch 2 Pack 12 Step 1920 Loss 2.8176515102386475
Epoch 2 Pack 12 Step 2048 Loss 4.848748207092285
Epoch 2 Pack 12 Step 2176 Loss 3.3521759510040283
Epoch 2 Pack 12 Step 2304 Loss 2.5843605995178223
Epoch 2 Pack 12 

Epoch 2 Pack 18 Step 2048 Loss 2.86169695854187
Epoch 2 Pack 18 Step 2176 Loss 3.118479013442993
Epoch 2 Pack 18 Step 2304 Loss 2.9309637546539307
Epoch 2 Pack 18 Step 2432 Loss 3.034343719482422
Epoch 2 Pack 18 Step 2560 Loss 2.863990545272827
Epoch 2 Pack 18 Step 2688 Loss 2.6163527965545654
Epoch 2 Pack 18 Step 2816 Loss 2.869821310043335
Epoch 2 Pack 18 Step 2944 Loss 2.9264466762542725
Epoch 2 Pack 18 Step 3072 Loss 3.027798652648926
Epoch 2 Pack 19 Step 0 Loss 2.9029462337493896
Epoch 2 Pack 19 Step 128 Loss 2.912825345993042
Epoch 2 Pack 19 Step 256 Loss 2.8363149166107178
Epoch 2 Pack 19 Step 384 Loss 2.9959189891815186
Epoch 2 Pack 19 Step 512 Loss 3.161968469619751
Epoch 2 Pack 19 Step 640 Loss 2.74526309967041
Epoch 2 Pack 19 Step 768 Loss 3.0157439708709717
Epoch 2 Pack 19 Step 896 Loss 2.420851945877075
Epoch 2 Pack 19 Step 1024 Loss 2.894408702850342
Epoch 2 Pack 19 Step 1152 Loss 2.950216293334961
Epoch 2 Pack 19 Step 1280 Loss 2.769400119781494
Epoch 2 Pack 19 Step 1408

Epoch 2 Pack 25 Step 1152 Loss 2.3198916912078857
Epoch 2 Pack 25 Step 1280 Loss 2.4962432384490967
Epoch 2 Pack 25 Step 1408 Loss 2.5237531661987305
Epoch 2 Pack 25 Step 1536 Loss 2.349687337875366
Epoch 2 Pack 25 Step 1664 Loss 2.237001657485962
Epoch 2 Pack 25 Step 1792 Loss 2.3795313835144043
Epoch 2 Pack 25 Step 1920 Loss 2.150937080383301
Epoch 2 Pack 25 Step 2048 Loss 2.175407886505127
Epoch 2 Pack 25 Step 2176 Loss 2.000788450241089
Epoch 2 Pack 25 Step 2304 Loss 2.1922857761383057
Epoch 2 Pack 25 Step 2432 Loss 2.3821985721588135
Epoch 2 Pack 25 Step 2560 Loss 2.464444875717163
Epoch 2 Pack 25 Step 2688 Loss 1.990659475326538
Epoch 2 Pack 25 Step 2816 Loss 2.3662383556365967
Epoch 2 Pack 25 Step 2944 Loss 2.3180768489837646
Epoch 2 Pack 25 Step 3072 Loss 2.542165994644165
Epoch 2 Pack 26 Step 0 Loss 2.314673662185669
Epoch 2 Pack 26 Step 128 Loss 2.3938333988189697
Epoch 2 Pack 26 Step 256 Loss 2.269887924194336
Epoch 2 Pack 26 Step 384 Loss 2.2159667015075684
Epoch 2 Pack 26 

Epoch 2 Pack 32 Step 128 Loss 2.133657932281494
Epoch 2 Pack 32 Step 256 Loss 2.1318953037261963
Epoch 2 Pack 32 Step 384 Loss 1.9908359050750732
Epoch 2 Pack 32 Step 512 Loss 2.3623175621032715
Epoch 2 Pack 32 Step 640 Loss 1.7856453657150269
Epoch 2 Pack 32 Step 768 Loss 2.0942928791046143
Epoch 2 Pack 32 Step 896 Loss 1.864120602607727
Epoch 2 Pack 32 Step 1024 Loss 2.037095546722412
Epoch 2 Pack 32 Step 1152 Loss 2.0641117095947266
Epoch 2 Pack 32 Step 1280 Loss 1.9402683973312378
Epoch 2 Pack 32 Step 1408 Loss 1.719662070274353
Epoch 2 Pack 32 Step 1536 Loss 2.155642032623291
Epoch 2 Pack 32 Step 1664 Loss 2.0930821895599365
Epoch 2 Pack 32 Step 1792 Loss 2.142493963241577
Epoch 2 Pack 32 Step 1920 Loss 2.143117904663086
Epoch 2 Pack 32 Step 2048 Loss 2.035503387451172
Epoch 2 Pack 32 Step 2176 Loss 2.00180721282959
Epoch 2 Pack 32 Step 2304 Loss 2.121060371398926
Epoch 2 Pack 32 Step 2432 Loss 2.1273841857910156
Epoch 2 Pack 32 Step 2560 Loss 1.9026737213134766
Epoch 2 Pack 32 St

Epoch 2 Pack 38 Step 2304 Loss 2.0459656715393066
Epoch 2 Pack 38 Step 2432 Loss 1.8080796003341675
Epoch 2 Pack 38 Step 2560 Loss 1.8540118932724
Epoch 2 Pack 38 Step 2688 Loss 2.227602481842041
Epoch 2 Pack 38 Step 2816 Loss 1.6732620000839233
Epoch 2 Pack 38 Step 2944 Loss 2.3420250415802
Epoch 2 Pack 38 Step 3072 Loss 2.0092897415161133
Epoch 2 Pack 39 Step 0 Loss 2.1626477241516113
Epoch 2 Pack 39 Step 128 Loss 2.065889596939087
Epoch 2 Pack 39 Step 256 Loss 2.2043325901031494
Epoch 2 Pack 39 Step 384 Loss 2.2289505004882812
Epoch 2 Pack 39 Step 512 Loss 2.1887011528015137
Epoch 2 Pack 39 Step 640 Loss 2.2378437519073486
Epoch 2 Pack 39 Step 768 Loss 2.220355272293091
Epoch 2 Pack 39 Step 896 Loss 2.3599319458007812
Epoch 2 Pack 39 Step 1024 Loss 2.3451550006866455
Epoch 2 Pack 39 Step 1152 Loss 2.4515440464019775
Epoch 2 Pack 39 Step 1280 Loss 2.380033016204834
Epoch 2 Pack 39 Step 1408 Loss 2.2203054428100586
Epoch 2 Pack 39 Step 1536 Loss 2.3618712425231934
Epoch 2 Pack 39 Step

In [7]:
torch.save(model.state_dict(), 'graph/2/model2')

In [24]:
model.load_state_dict(torch.load('graph/2/model2'))

<All keys matched successfully>

In [25]:
optimizer = torch.optim.Adam(model.parameters(),  betas=(0.9, 0.98), eps=1e-9, lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_EN)

for epoch in range(2):
    for pack in range(45):
        with open('data/train/' + str(pack) + '.pkl', 'rb') as f: 
            X = pkl.load(f)
        dloader = DataLoader(X, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=generate_batch)
        for i, (seq1, seq2) in enumerate(dloader):
            tgt_in = seq2[:-1]
            pred = model(seq1, tgt_in)

            tgt_out = seq2[1:]
            loss = criterion(pred.reshape(-1, 16386), tgt_out.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 128 == 0:
                print('Epoch {} Pack {} Step {} Loss {}'.format(epoch, pack, i, loss.detach()))
        torch.save(model.state_dict(), 'graph/2/model3')            

Epoch 0 Pack 0 Step 0 Loss 3.5919876098632812
Epoch 0 Pack 0 Step 128 Loss 2.9436941146850586
Epoch 0 Pack 0 Step 256 Loss 2.769303560256958
Epoch 0 Pack 0 Step 384 Loss 3.0139288902282715
Epoch 0 Pack 0 Step 512 Loss 2.769371747970581
Epoch 0 Pack 0 Step 640 Loss 3.0694875717163086
Epoch 0 Pack 0 Step 768 Loss 2.7923777103424072
Epoch 0 Pack 0 Step 896 Loss 2.8123574256896973
Epoch 0 Pack 0 Step 1024 Loss 2.477907180786133
Epoch 0 Pack 0 Step 1152 Loss 2.502678394317627
Epoch 0 Pack 0 Step 1280 Loss 2.754563808441162
Epoch 0 Pack 0 Step 1408 Loss 2.8062644004821777
Epoch 0 Pack 0 Step 1536 Loss 2.6509246826171875
Epoch 0 Pack 0 Step 1664 Loss 3.038620710372925
Epoch 0 Pack 0 Step 1792 Loss 2.478189706802368
Epoch 0 Pack 0 Step 1920 Loss 2.338650703430176
Epoch 0 Pack 0 Step 2048 Loss 2.6049580574035645
Epoch 0 Pack 0 Step 2176 Loss 2.4317433834075928
Epoch 0 Pack 0 Step 2304 Loss 2.536740779876709
Epoch 0 Pack 0 Step 2432 Loss 2.4504778385162354
Epoch 0 Pack 0 Step 2560 Loss 2.5852391

Epoch 0 Pack 6 Step 2688 Loss 1.9845212697982788
Epoch 0 Pack 6 Step 2816 Loss 2.1151695251464844
Epoch 0 Pack 6 Step 2944 Loss 2.132932186126709
Epoch 0 Pack 6 Step 3072 Loss 2.2732126712799072
Epoch 0 Pack 7 Step 0 Loss 2.156534433364868
Epoch 0 Pack 7 Step 128 Loss 2.6154918670654297
Epoch 0 Pack 7 Step 256 Loss 2.61283016204834
Epoch 0 Pack 7 Step 384 Loss 2.6274049282073975
Epoch 0 Pack 7 Step 512 Loss 2.7165653705596924
Epoch 0 Pack 7 Step 640 Loss 2.548285961151123
Epoch 0 Pack 7 Step 768 Loss 2.5236659049987793
Epoch 0 Pack 7 Step 896 Loss 2.486403465270996
Epoch 0 Pack 7 Step 1024 Loss 2.6333847045898438
Epoch 0 Pack 7 Step 1152 Loss 2.5511627197265625
Epoch 0 Pack 7 Step 1280 Loss 2.403883218765259
Epoch 0 Pack 7 Step 1408 Loss 2.733722448348999
Epoch 0 Pack 7 Step 1536 Loss 2.595146417617798
Epoch 0 Pack 7 Step 1664 Loss 2.527324676513672
Epoch 0 Pack 7 Step 1792 Loss 2.6379659175872803
Epoch 0 Pack 7 Step 1920 Loss 2.26483154296875
Epoch 0 Pack 7 Step 2048 Loss 2.8493688106

Epoch 0 Pack 13 Step 2048 Loss 2.3584420680999756
Epoch 0 Pack 13 Step 2176 Loss 2.372255325317383
Epoch 0 Pack 13 Step 2304 Loss 2.386996030807495
Epoch 0 Pack 13 Step 2432 Loss 2.2139031887054443
Epoch 0 Pack 13 Step 2560 Loss 2.384542226791382
Epoch 0 Pack 13 Step 2688 Loss 1.8095084428787231
Epoch 0 Pack 13 Step 2816 Loss 2.629669189453125
Epoch 0 Pack 13 Step 2944 Loss 2.583881139755249
Epoch 0 Pack 13 Step 3072 Loss 2.298628568649292
Epoch 0 Pack 14 Step 0 Loss 2.508760690689087
Epoch 0 Pack 14 Step 128 Loss 2.0811784267425537
Epoch 0 Pack 14 Step 256 Loss 2.2866508960723877
Epoch 0 Pack 14 Step 384 Loss 2.351569652557373
Epoch 0 Pack 14 Step 512 Loss 2.277939558029175
Epoch 0 Pack 14 Step 640 Loss 2.2414331436157227
Epoch 0 Pack 14 Step 768 Loss 2.5530033111572266
Epoch 0 Pack 14 Step 896 Loss 2.577177047729492
Epoch 0 Pack 14 Step 1024 Loss 2.287555456161499
Epoch 0 Pack 14 Step 1152 Loss 2.21820330619812
Epoch 0 Pack 14 Step 1280 Loss 2.3952646255493164
Epoch 0 Pack 14 Step 14

Epoch 0 Pack 20 Step 1024 Loss 2.221060276031494
Epoch 0 Pack 20 Step 1152 Loss 2.6666440963745117
Epoch 0 Pack 20 Step 1280 Loss 2.323733329772949
Epoch 0 Pack 20 Step 1408 Loss 1.9883075952529907
Epoch 0 Pack 20 Step 1536 Loss 2.200566291809082
Epoch 0 Pack 20 Step 1664 Loss 2.5810556411743164
Epoch 0 Pack 20 Step 1792 Loss 2.130652904510498
Epoch 0 Pack 20 Step 1920 Loss 3.069622755050659
Epoch 0 Pack 20 Step 2048 Loss 2.520271062850952
Epoch 0 Pack 20 Step 2176 Loss 2.2553763389587402
Epoch 0 Pack 20 Step 2304 Loss 2.1112163066864014
Epoch 0 Pack 20 Step 2432 Loss 2.481992721557617
Epoch 0 Pack 20 Step 2560 Loss 2.4675941467285156
Epoch 0 Pack 20 Step 2688 Loss 2.4673588275909424
Epoch 0 Pack 20 Step 2816 Loss 2.19292950630188
Epoch 0 Pack 20 Step 2944 Loss 2.0701677799224854
Epoch 0 Pack 20 Step 3072 Loss 2.3541371822357178
Epoch 0 Pack 21 Step 0 Loss 2.5726892948150635
Epoch 0 Pack 21 Step 128 Loss 2.392017364501953
Epoch 0 Pack 21 Step 256 Loss 2.3377254009246826
Epoch 0 Pack 21

Epoch 0 Pack 27 Step 0 Loss 1.669593334197998
Epoch 0 Pack 27 Step 128 Loss 1.6836977005004883
Epoch 0 Pack 27 Step 256 Loss 1.6490825414657593
Epoch 0 Pack 27 Step 384 Loss 1.6295980215072632
Epoch 0 Pack 27 Step 512 Loss 1.7100193500518799
Epoch 0 Pack 27 Step 640 Loss 1.5252516269683838
Epoch 0 Pack 27 Step 768 Loss 1.5312424898147583
Epoch 0 Pack 27 Step 896 Loss 1.5391653776168823
Epoch 0 Pack 27 Step 1024 Loss 1.6720890998840332
Epoch 0 Pack 27 Step 1152 Loss 1.7939752340316772
Epoch 0 Pack 27 Step 1280 Loss 1.5694572925567627
Epoch 0 Pack 27 Step 1408 Loss 1.5910109281539917
Epoch 0 Pack 27 Step 1536 Loss 2.033107280731201
Epoch 0 Pack 27 Step 1664 Loss 1.495279312133789
Epoch 0 Pack 27 Step 1792 Loss 1.8495393991470337
Epoch 0 Pack 27 Step 1920 Loss 1.6006903648376465
Epoch 0 Pack 27 Step 2048 Loss 1.5763230323791504
Epoch 0 Pack 27 Step 2176 Loss 1.7091518640518188
Epoch 0 Pack 27 Step 2304 Loss 1.7134701013565063
Epoch 0 Pack 27 Step 2432 Loss 1.7475026845932007
Epoch 0 Pack 

...

Epoch 1 Pack 15 Step 3072 Loss 2.534909725189209
Epoch 1 Pack 16 Step 0 Loss 2.6732795238494873
Epoch 1 Pack 16 Step 128 Loss 1.6706640720367432
Epoch 1 Pack 16 Step 256 Loss 2.4510374069213867
Epoch 1 Pack 16 Step 384 Loss 2.280974864959717
Epoch 1 Pack 16 Step 512 Loss 1.7510416507720947
Epoch 1 Pack 16 Step 640 Loss 2.2013208866119385
Epoch 1 Pack 16 Step 768 Loss 2.646078586578369
Epoch 1 Pack 16 Step 896 Loss 2.4693498611450195
Epoch 1 Pack 16 Step 1024 Loss 2.2676403522491455
Epoch 1 Pack 16 Step 1152 Loss 2.218729019165039
Epoch 1 Pack 16 Step 1280 Loss 2.4584097862243652
Epoch 1 Pack 16 Step 1408 Loss 2.436997890472412
Epoch 1 Pack 16 Step 1536 Loss 2.5223824977874756
Epoch 1 Pack 16 Step 1664 Loss 1.85158109664917
Epoch 1 Pack 16 Step 1792 Loss 2.8831684589385986
Epoch 1 Pack 16 Step 1920 Loss 2.4857771396636963
Epoch 1 Pack 16 Step 2048 Loss 2.2322051525115967
Epoch 1 Pack 16 Step 2176 Loss 2.4310874938964844
Epoch 1 Pack 16 Step 2304 Loss 2.315009355545044
Epoch 1 Pack 16 St

Epoch 1 Pack 22 Step 2048 Loss 1.7365405559539795
Epoch 1 Pack 22 Step 2176 Loss 2.2130181789398193
Epoch 1 Pack 22 Step 2304 Loss 2.2584896087646484
Epoch 1 Pack 22 Step 2432 Loss 2.5236477851867676
Epoch 1 Pack 22 Step 2560 Loss 2.031167507171631
Epoch 1 Pack 22 Step 2688 Loss 2.5065712928771973
Epoch 1 Pack 22 Step 2816 Loss 2.268381357192993
Epoch 1 Pack 22 Step 2944 Loss 2.197589635848999
Epoch 1 Pack 22 Step 3072 Loss 2.5376176834106445
Epoch 1 Pack 23 Step 0 Loss 2.6196441650390625
Epoch 1 Pack 23 Step 128 Loss 2.8191490173339844
Epoch 1 Pack 23 Step 256 Loss 2.468442678451538
Epoch 1 Pack 23 Step 384 Loss 2.54906964302063
Epoch 1 Pack 23 Step 512 Loss 2.128671169281006
Epoch 1 Pack 23 Step 640 Loss 2.532128095626831
Epoch 1 Pack 23 Step 768 Loss 2.47343373298645
Epoch 1 Pack 23 Step 896 Loss 2.5952818393707275
Epoch 1 Pack 23 Step 1024 Loss 2.411902666091919
Epoch 1 Pack 23 Step 1152 Loss 2.119903326034546
Epoch 1 Pack 23 Step 1280 Loss 2.3904826641082764
Epoch 1 Pack 23 Step 1

Epoch 1 Pack 29 Step 1024 Loss 1.604319453239441
Epoch 1 Pack 29 Step 1152 Loss 1.478427529335022
Epoch 1 Pack 29 Step 1280 Loss 1.3250874280929565
Epoch 1 Pack 29 Step 1408 Loss 1.2829678058624268
Epoch 1 Pack 29 Step 1536 Loss 1.4502575397491455
Epoch 1 Pack 29 Step 1664 Loss 1.506453275680542
Epoch 1 Pack 29 Step 1792 Loss 1.8477014303207397
Epoch 1 Pack 29 Step 1920 Loss 1.4554857015609741
Epoch 1 Pack 29 Step 2048 Loss 1.4538179636001587
Epoch 1 Pack 29 Step 2176 Loss 1.577972173690796
Epoch 1 Pack 29 Step 2304 Loss 1.417436122894287
Epoch 1 Pack 29 Step 2432 Loss 1.5518512725830078
Epoch 1 Pack 29 Step 2560 Loss 1.327481985092163
Epoch 1 Pack 29 Step 2688 Loss 1.390648603439331
Epoch 1 Pack 29 Step 2816 Loss 1.3011354207992554
Epoch 1 Pack 29 Step 2944 Loss 1.5094102621078491
Epoch 1 Pack 29 Step 3072 Loss 1.6049203872680664
Epoch 1 Pack 30 Step 0 Loss 1.6065763235092163
Epoch 1 Pack 30 Step 128 Loss 1.540299654006958
Epoch 1 Pack 30 Step 256 Loss 1.5245522260665894
Epoch 1 Pack 

Epoch 1 Pack 35 Step 3072 Loss 0.9630632996559143
Epoch 1 Pack 36 Step 0 Loss 1.0549132823944092
Epoch 1 Pack 36 Step 128 Loss 1.3763694763183594
Epoch 1 Pack 36 Step 256 Loss 1.1910176277160645
Epoch 1 Pack 36 Step 384 Loss 1.2415621280670166
Epoch 1 Pack 36 Step 512 Loss 1.323897123336792
Epoch 1 Pack 36 Step 640 Loss 1.1544122695922852
Epoch 1 Pack 36 Step 768 Loss 1.2059129476547241
Epoch 1 Pack 36 Step 896 Loss 1.2398353815078735
Epoch 1 Pack 36 Step 1024 Loss 1.3617132902145386
Epoch 1 Pack 36 Step 1152 Loss 1.231474757194519
Epoch 1 Pack 36 Step 1280 Loss 0.9751068353652954
Epoch 1 Pack 36 Step 1408 Loss 1.0173993110656738
Epoch 1 Pack 36 Step 1536 Loss 1.1465567350387573
Epoch 1 Pack 36 Step 1664 Loss 1.1374307870864868
Epoch 1 Pack 36 Step 1792 Loss 1.2200225591659546
Epoch 1 Pack 36 Step 1920 Loss 1.0401583909988403
Epoch 1 Pack 36 Step 2048 Loss 1.268970012664795
Epoch 1 Pack 36 Step 2176 Loss 1.1498308181762695
Epoch 1 Pack 36 Step 2304 Loss 1.1090507507324219
Epoch 1 Pack 

Epoch 1 Pack 42 Step 1920 Loss 1.6432024240493774
Epoch 1 Pack 42 Step 2048 Loss 1.5858312845230103
Epoch 1 Pack 42 Step 2176 Loss 1.6294177770614624
Epoch 1 Pack 42 Step 2304 Loss 1.5697858333587646
Epoch 1 Pack 42 Step 2432 Loss 1.4517691135406494
Epoch 1 Pack 42 Step 2560 Loss 1.6264948844909668
Epoch 1 Pack 42 Step 2688 Loss 1.244839072227478
Epoch 1 Pack 42 Step 2816 Loss 1.6643942594528198
Epoch 1 Pack 42 Step 2944 Loss 1.491481065750122
Epoch 1 Pack 42 Step 3072 Loss 1.5995101928710938
Epoch 1 Pack 43 Step 0 Loss 1.855484127998352
Epoch 1 Pack 43 Step 128 Loss 1.6167817115783691
Epoch 1 Pack 43 Step 256 Loss 1.631355881690979
Epoch 1 Pack 43 Step 384 Loss 1.5186052322387695
Epoch 1 Pack 43 Step 512 Loss 1.681949496269226
Epoch 1 Pack 43 Step 640 Loss 1.3411738872528076
Epoch 1 Pack 43 Step 768 Loss 1.349867582321167
Epoch 1 Pack 43 Step 896 Loss 1.407902717590332
Epoch 1 Pack 43 Step 1024 Loss 1.4815292358398438
Epoch 1 Pack 43 Step 1152 Loss 1.2255131006240845
Epoch 1 Pack 43 S

In [26]:
torch.save(model.state_dict(), 'graph/2/model4')

# Demo

In [11]:
import torch
from model import gen_mask
BOS_IDX = torch.tensor(BOS_EN).view(-1, 1)
EOS_IDX = torch.tensor(EOS_EN).view(-1, 1)

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to('cuda')
    src_mask = src_mask.to('cuda')

    memory = model.encode(src, src_mask)
    ys = start_symbol.type(torch.long).to('cuda')
    for i in range(max_len - 1):
        memory = memory.to('cuda')
        tgt_mask = (gen_mask(ys.size(0))
                    .type(torch.bool)).to('cuda')
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.fc(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
    return ys


def translate(model, src_sentence):
    model.eval()
    src = torch.cat([torch.tensor([BOS_DE]), 
                     torch.tensor(next(de_vocab.transform([seq]))),
                     torch.tensor([EOS_DE])], dim=0).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, 
                               max_len=num_tokens + 40, 
                               start_symbol=BOS_IDX).flatten()
    return tgt_tokens

In [61]:
seq = 'Ohne dich kann ich nicht sein, ohne dich .'

tgt_tokens = translate(model, seq)

In [62]:
end = min((tgt_tokens == 16385).nonzero()[0][0] if torch.sum((tgt_tokens == 16385)) else tgt_tokens.shape[0], 
          tgt_tokens.shape[0])
tmp = [[x.item() for x in tgt_tokens[1:end]]]
next(en_vocab.inverse_transform(tmp)).replace(' & apos ; ', "'")

'without you , i can ’ t be without you , without you .'